**Final Report: Performance Analysis and Ranking of Finance Accounts**

---

## **1. Introduction**
This report presents a detailed analysis of historical trade data from various Binance accounts over a 90-day period. The goal of this analysis is to compute key financial metrics for each account, rank them based on performance, and provide a list of the top 20 accounts.

## **2. Data Preprocessing**
### **2.1 Dataset Overview**
The dataset consists of:
- **Port_IDs**: Unique identifier for each account.
- **Trade_History**: A JSON-like structure containing trade details, including timestamp, asset, side (BUY/SELL), price, quantity, and realized profit/loss.

### **2.2 Data Cleaning and Parsing**
- Missing values in `Trade_History` were identified and removed.
- The `Trade_History` column was converted from a JSON-like string into a structured format.
- Key trade details for each account were extracted into separate DataFrames for further analysis.

## **3. Financial Metrics Calculation**
For each account, the following financial metrics were computed:

### **3.1 Return on Investment (ROI)**
\[ ROI = \frac{\text{Net Profit}}{\text{Total Investment}} \times 100 \]
- **Net Profit**: Sum of `realizedProfit` across all trades.
- **Total Investment**: Sum of `quantity` used in all trades.

### **3.2 Profit and Loss (PnL)**
\[ PnL = \sum \text{realizedProfit} \]
- Measures the total profitability or loss of an account.

### **3.3 Sharpe Ratio**
\[ Sharpe Ratio = \frac{\text{Mean Returns}}{\text{Standard Deviation of Returns}} \]
- Measures risk-adjusted returns. A higher Sharpe Ratio indicates better returns per unit of risk.

### **3.4 Maximum Drawdown (MDD)**
\[ MDD = \frac{\text{Peak Portfolio Value} - \text{Lowest Portfolio Value}}{\text{Peak Portfolio Value}} \]
- Represents the worst percentage drop in portfolio value from peak to trough.

### **3.5 Win Rate & Positions**
- **Win Rate** = Percentage of trades that resulted in a profit.
- **Win Positions** = Number of profitable trades.
- **Total Positions** = Total number of trades executed.

## **4. Ranking Methodology**
Each account was ranked based on a weighted scoring system:
\[ Score = 0.4 \times ROI + 0.2 \times Sharpe Ratio + 0.2 \times Win Rate + 0.1 \times PnL - 0.1 \times MDD \]

- **ROI (40%)**: Higher ROI is preferred.
- **Sharpe Ratio (20%)**: Penalizes volatile returns.
- **Win Rate (20%)**: Measures consistency in profitability.
- **PnL (10%)**: Prioritizes accounts with higher profits.
- **MDD (-10%)**: Penalizes accounts with higher drawdowns.

The final rankings were determined based on the computed scores, and the top 20 accounts were selected.

## **5. Findings & Insights**
- Accounts with high ROI and Sharpe Ratios generally performed well.
- Some accounts had high win rates but lower PnL, indicating small but frequent profits.
- Accounts with significant drawdowns (high MDD) were penalized in rankings due to increased risk.
- A balanced approach between profitability and risk-adjusted returns led to better ranking scores.

## **6. Deliverables**
- **Jupyter Notebook/Python Script**: Contains the full analysis and computations.
- **Calculated Metrics CSV (calculated_metrics.csv)**: Includes all computed metrics for each account.
- **Top 20 Accounts CSV (top_20_accounts.csv)**: Lists the top 20 ranked accounts.
- **This Final Report**: Summarizes methodology, findings, and assumptions.

---

This report provides a comprehensive assessment of Binance trading accounts, identifying top-performing traders based on quantitative metrics and ranking methodology.



In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
data = pd.read_csv(r'E:\Data Set\csv file\TRADES_CopyTr_90D_ROI.csv')

In [5]:
data.head(5)

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [9]:
data.shape

(150, 2)

In [11]:
data.isnull().sum()

Port_IDs         0
Trade_History    1
dtype: int64

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [17]:
data.dtypes

Port_IDs          int64
Trade_History    object
dtype: object

# Handing Missing Values 

In [24]:
data = data.dropna(subset=['Trade_History'])

In [26]:
data.isnull().sum()

Port_IDs         0
Trade_History    0
dtype: int64

In [28]:
import ast
data['Trade_History'] = data['Trade_History'].apply(ast.literal_eval)


In [30]:
data.head(3)

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."


In [95]:
data['Trade_History'][0]

[{'time': 1718899656000,
  'symbol': 'SOLUSDT',
  'side': 'BUY',
  'price': 132.537,
  'fee': -0.9940275,
  'feeAsset': 'USDT',
  'quantity': 1988.055,
  'quantityAsset': 'USDT',
  'realizedProfit': 0.0,
  'realizedProfitAsset': 'USDT',
  'baseAsset': 'SOL',
  'qty': 15.0,
  'positionSide': 'LONG',
  'activeBuy': True},
 {'time': 1718899618000,
  'symbol': 'DOGEUSDT',
  'side': 'BUY',
  'price': 0.12182,
  'fee': -0.27979617,
  'feeAsset': 'USDT',
  'quantity': 1398.98088,
  'quantityAsset': 'USDT',
  'realizedProfit': 0.0,
  'realizedProfitAsset': 'USDT',
  'baseAsset': 'DOGE',
  'qty': 11484.0,
  'positionSide': 'LONG',
  'activeBuy': False},
 {'time': 1718899618000,
  'symbol': 'DOGEUSDT',
  'side': 'BUY',
  'price': 0.12182,
  'fee': -0.03949404,
  'feeAsset': 'USDT',
  'quantity': 197.47022,
  'quantityAsset': 'USDT',
  'realizedProfit': 0.0,
  'realizedProfitAsset': 'USDT',
  'baseAsset': 'DOGE',
  'qty': 1621.0,
  'positionSide': 'LONG',
  'activeBuy': False},
 {'time': 17188996

In [38]:
# Displaying sample parsed trade history
print(data['Trade_History'].iloc[0][:2])

[{'time': 1718899656000, 'symbol': 'SOLUSDT', 'side': 'BUY', 'price': 132.537, 'fee': -0.9940275, 'feeAsset': 'USDT', 'quantity': 1988.055, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'SOL', 'qty': 15.0, 'positionSide': 'LONG', 'activeBuy': True}, {'time': 1718899618000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.27979617, 'feeAsset': 'USDT', 'quantity': 1398.98088, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 11484.0, 'positionSide': 'LONG', 'activeBuy': False}]


# Feature Engineering

In [61]:

# Function to calculate financial metrics
def calculate_metrics(trades):
    # Convert trade history into DataFrame
    trades_data = pd.DataFrame(trades)
    
    # Ensure relevant columns exist
    if 'realizedProfit' not in trades_data.columns or 'quantity' not in trades_data.columns:
        return None  # Skip accounts with missing data

    # Calculating total profit and loss (PnL)
    total_pnl = trades_data['realizedProfit'].sum()
    
    # Calculating ROI (Return on Investment)
    total_investment = trades_data['quantity'].sum()
    roi = (total_pnl / total_investment) * 100 if total_investment != 0 else 0

    # Calculating win rate
    win_positions = (trades_data['realizedProfit'] > 0).sum()
    total_positions = len(trades_data)
    win_rate = (win_positions / total_positions) * 100 if total_positions != 0 else 0

    # Calculating Sharpe Ratio
    returns = trades_data['realizedProfit']
    if len(returns) > 1:
        sharpe_ratio = np.mean(returns) / np.std(returns) if np.std(returns) != 0 else 0
    else:
        sharpe_ratio = 0

    # Calculating Maximum Drawdown (MDD)
    cumulative_pnl = trades_data['realizedProfit'].cumsum()
    peak = cumulative_pnl.cummax()
    drawdown = peak - cumulative_pnl
    max_drawdown = drawdown.max()

    return {
        'ROI': roi,
        'PnL': total_pnl,
        'Sharpe_Ratio': sharpe_ratio,
        'MDD': max_drawdown,
        'Win_Rate': win_rate,
        'Win_Positions': win_positions,
        'Total_Positions': total_positions
    }


In [63]:

# Applying function to all accounts
metrics_list = []
for _, row in data.iterrows():
    metrics = calculate_metrics(row['Trade_History'])
    if metrics:
        metrics['Port_ID'] = row['Port_IDs']
        metrics_list.append(metrics)

# Creating new DataFrame
metrics_data = pd.DataFrame(metrics_list)

# Displaying the  results
print(metrics_data.head())


        ROI          PnL  Sharpe_Ratio          MDD   Win_Rate  Win_Positions  \
0  0.375303  6789.436739      0.274968   162.057536  33.333333            486   
1  2.492178  3686.969761      0.125892    38.336310  18.576389            107   
2  0.156489   772.729032      0.072410   205.532658  28.514694            359   
3  0.828347  3658.765062      0.119475   175.481291  55.283308            361   
4  0.135459  1309.459960      0.050312  1119.093200  34.100418            163   

   Total_Positions              Port_ID  
0             1458  3925368433214965504  
1              576  4002413037164645377  
2             1259  3923766029921022977  
3              653  3994879592543698688  
4              478  3926423286576838657  


# Creating Ranking Algorithm 

In [70]:
# Defined weights for ranking
weights = {
    'ROI': 0.4,
    'Sharpe_Ratio': 0.2,
    'Win_Rate': 0.2,
    'PnL': 0.1,
    'MDD': -0.1


In [72]:
# Compute final ranking score
metrics_data['Score'] = (
    metrics_data['ROI'] * weights['ROI'] +
    metrics_data['Sharpe_Ratio'] * weights['Sharpe_Ratio'] +
    metrics_data['Win_Rate'] * weights['Win_Rate'] +
    metrics_data['PnL'] * weights['PnL'] +
    metrics_data['MDD'] * weights['MDD']
)


In [74]:
# Rank and get top 20 accounts
top_20 = metrics_data.sort_values(by='Score', ascending=False).head(20)

In [86]:

# Saviing results
metrics_data.to_csv("calculated_metrics.csv", index=False)
top_20.to_csv("top_20_accounts.csv", index=False)

In [82]:
# Display top 20
print(top_20)


          ROI           PnL  Sharpe_Ratio           MDD   Win_Rate  \
82   0.311561  71998.855953      0.060753  21338.051188  32.942149   
126  0.543673  42574.473679      0.227530    300.813848  52.321981   
23   0.404420  26427.331592      0.068271   2537.866629  31.391201   
64   0.233296  19567.471286      0.162982   2234.000000  51.568381   
146  0.406480  17601.401398      0.275925    585.677600  54.294032   
35   0.871628  16337.461881      0.329833      2.381947  78.221722   
22   1.032230  16790.012238      0.099541    635.050015  51.557712   
118  1.099242  18015.997370      0.222756   2060.707122  61.397148   
87   0.233080  14197.577076      0.299815    385.075554  52.765957   
43   0.836806  13960.966457      0.152937   1772.219440  44.607364   
107  0.308240  11298.706390      0.266628    509.550000  42.131980   
60   1.791054  10681.514000      0.306751      0.000000  30.000000   
75   1.016280   9775.389110      0.204154     87.003360  37.460317   
25   0.468031   9603